Population Spread in London City Boroughs

**Imports**

In [1]:
import csv as csv 
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import geopandas as gp
import statsmodels.api as sm

**Loading of datasets**

In [2]:
#Loading of my datasets into the console
employmentrate = pd.read_excel('../input/employment-rate/employmentrate2017.xlsx')

employmentrate.head()

#References
#London DataStore
#(Employment Rates by Ethnicity)
#(https://data.london.gov.uk/dataset/employment-rates-by-ethnicity)

In [3]:
#Loading of my second dataset
employmentindustry = pd.read_excel('../input/employment-industry/employmentindustry2017.xlsx')

employmentindustry.head()

#References
#London DataStore
#(Workplace Employment by Industry
#(https://data.london.gov.uk/dataset/workplace-employment-industry-borough))

In [4]:
#Loading of my third dataset
londoncensus = pd.read_csv('../input/london-census/londoncensus.csv')

londoncensus.head()

#References
#London Census Data
#https://moodle.city.ac.uk/mod/page/view.php?id=1400847

**Dataset Information**

In [5]:
#Datasets Information, mean, max, std for example
print('The Employment Rate dataframe has {} rows and {} columns'.format(employmentrate.shape[0], employmentrate.shape[1]))
print('The Employment Industry dataframe has {} rows and {} columns'.format(employmentindustry.shape[0], employmentindustry.shape[1]))
print('The London Census dataframe has {} rows and {} columns'.format(londoncensus.shape[0], londoncensus.shape[1]))


**Description of Datasets**

In [6]:
employmentrate.describe(include='all')

In [7]:
employmentindustry.describe(include='all')

In [8]:
londoncensus.describe(include='all')

**Cleaning the datasets**

I used OpenRefine to clean all the datasets, mostly the employmentrate and employment industry. For the londoncensus data, the only data wrangling I did was to change all the column names related to boroughs in London.
In the employmentrate dataset there are three missing values in two columns with an exclamation point '!'. One in the 'Indian' column and two in the 'Paskistan/Bangladesh' column. Also in Lambeth, the percentage is 100% for Pakistan/Bangladesh when in actuality, there was't any data for number of employed divided by total population.
Below, I used different techniques to locate the three missing data in the employmentrate dataset and then replaced them with the means of the two columns with missing data.

**Filling out the missing datasets**

In [9]:
#I tried using dataframes shape and count to give me the number of missing values
#But it didn't show me the 3 missing values with exclamation point (!) in the employmentrate dataset
print (employmentrate.shape[0] - employmentrate.count())

In [10]:
#Then I tried using isnull() to find out if the 3 missing values would be discovered and it wasn't...
atLeastOneNaN = employmentrate['Indian'].isnull() | employmentrate['Pakistan/Bangladesh'].isnull()
print(atLeastOneNaN)

In [11]:
#Here are the three missing values in the employmentrate dataset
print(employmentrate.Indian[12])
print(employmentrate['Pakistan/Bangladesh'][21])
print(employmentrate['Pakistan/Bangladesh'][25])

In [12]:
#As you can see when I try to get the mean of the dataset, the two obvious columns with '!' in them are missing
employmentrate.mean()

In [13]:
#Right here, I am replacing the '!' with NaN values and then replacing those NaN values with the means of the....
#...'Indian' & 'Pakistan/Bangladesh' columns respectively.
employmentrate['Indian'] = employmentrate['Indian'].replace('!',np.NaN)
employmentrate['Indian'] = employmentrate['Indian'].fillna(employmentrate['Indian'].mean())

employmentrate['Pakistan/Bangladesh'] = employmentrate['Pakistan/Bangladesh'].replace('!',np.NaN)
employmentrate['Pakistan/Bangladesh'] = employmentrate['Pakistan/Bangladesh'].fillna(employmentrate['Pakistan/Bangladesh'].mean())

In [14]:
#Now when I compute the mean, it gives me the mean for all the columns in the employmentrate dataset
employmentrate.mean()

**Population Spread in London**

Now I will visually display the population spread in London across various boroughs. I am confident that questions like, 'What is the most populated area in London', 'Why is that area highly populated', 'Is the employment rate high/low in the most populated area in London', 'What employment industries have the highest employment in the most populated area in London' will be answered.

In [15]:
#First of all, I will slice the London Census data to get the columns I need to be analysed visually
#I will slice the dataset to get the rows that have all the boroughs and population by Age, Ethnic group, Sex,
#...Religion, and health condition columns
ldncensusdf = londoncensus.iloc[25:, [0, 1, 2, 3, 6, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 
                                      37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
                                      51, 54, 55, 56, 57, 58, 59, 60, 64, 65, 66, 67, 68]]
ldncensusdf.head()

In [16]:
#Loading the shp file for the london census data
ldnwards = gp.read_file('../input/london-shape-files/london_wards_2011_wgs84.shp')
ldnwards.head()

In [17]:
#Ploting georgraphical boundaries of london wards
ldnwards.plot(column='NAME', figsize=(40,10))
plt.show()

In [18]:
#Joining sliced london census data with geography
frames=[ldnwards,ldncensusdf]
data=pd.concat(frames,axis=1)
#Removing any wards that have missing values
ldngeo = data[data['Borough'].notnull()];
ldngeo.describe()

In [19]:
#Geographically plotting the cesus data to discover different spread across london based on Sex, Religion and...
#Health condition for example
ldngeo.crs = {'init' :'epsg:4326'}
attr = "No of all residents"
ax = ldngeo.plot(column=attr, cmap='YlGn', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by sex: Male"
ax = ldngeo.plot(column=attr, cmap='BuPu', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by sex: Female"
ax = ldngeo.plot(column=attr, cmap='YlOrBr', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by ethnic group: White"
ax = ldngeo.plot(column=attr, cmap='GnBu', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by religion: Jewish"
ax = ldngeo.plot(column=attr, cmap='Oranges', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by religion: Christian"
ax = ldngeo.plot(column=attr, cmap='Oranges', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by religion: Muslim"
ax = ldngeo.plot(column=attr, cmap='YlOrRd', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by health condition: Bad health"
ax = ldngeo.plot(column=attr, cmap='YlOrRd', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()
attr = "Population by health condition: Good health"
ax = ldngeo.plot(column=attr, cmap='YlOrRd', legend=True, figsize=(12,5))
ax.set_title(attr)
plt.show()

I geographically plotted the main variables(Ages, Ethnic group, Sex, Religion and Health condition) above. As you can see above, in the plot for the Jewish population in London, they are mainly located in the north of london where as the Christians are evenly spread across london. Residents with 'Bad Health' are almost pretty much evenly spread around London except south east london which is quite alarming to be honest.

In [20]:
#Using a hexbin plot with marginal distributions to distintly assess the spread of Jewish religion compared to 
#Christians
#Plotting the hexbin plot for Christians
sns.set(style="ticks")


x = ldncensusdf['No of all residents']
y = ldncensusdf['Population by religion: Christian']

sns.jointplot(x, y, kind="hex", color="#4CB391")
#Plotting the hexbin box plot for Jewish
sns.set(style="ticks")


x = ldncensusdf['No of all residents']
y = ldncensusdf['Population by religion: Jewish']

sns.jointplot(x, y, kind="hex", color="#4CB391")

[](http://)The Hexbin plot with marginal distributions reiterated my discovery that the Christian population are evenly distributed than the Jewish across london boroughs.

**Employment Rate**

Plotting the employment rate by ethnicity should be straight forward. I will cross check with my plotted geographical data to find out, for example, what religion is highly populated in that area or if the health condition of that area is good or bad. I used a horizontal bar chart because it lists all 36 boroughs on the vertical axis clearly.

In [21]:
#Using a horizontal bar chat
#Ploting the employment rate of White
plt.barh(employmentrate.Area, employmentrate.White, label='White')

#Ploting the employment rate of Mixed
plt.barh(employmentrate.Area, employmentrate.Mixed, label='Mixed')

#Ploting the employment rate of Indian
plt.barh(employmentrate.Area, employmentrate.Indian, label='Indian')

#Adding legend and labels
plt.legend(loc='center right', prop={'size': 15})
plt.title('Employment Rates by Ethnic Groups')
plt.xlabel('Percentages')
plt.ylabel('Boroughs')
plt.rcParams['figure.figsize'] = (30,20)

#Displaying the plot
plt.show()

I plotted 3 ethnic groups first so that it would be easier to discover what other ethnic group had the highest employment rate in each of the boroughs. Based on the bar chart above, White ethnic group are very highly to be employed in Harrow, being the highest, and Newham, Southwark and Wandsworth trailing very closely. Indians are likely to be employed in Camden, with Islington and Merton trailing very closely. Mixed ethnicities are likely to employed in Lambeth with only one other borough, Sutton, trailing closely behind.

In [22]:
#Using a horizontal bar chat
#Ploting the employment rate of Pakistan/Bangladesh
plt.barh(employmentrate.Area, employmentrate['Pakistan/Bangladesh'], label='Pakistan/Bangladesh', color='#86bf91')

#Ploting the employment rate of Black/Black British
plt.barh(employmentrate.Area, employmentrate['Black/Black British'], label='Black/Black British', color='#000000')
        
#Ploting the employment rate of Other ethnic groups
plt.barh(employmentrate.Area, employmentrate.Other, label='Other', color='#FFFF66')

#Adding legends and labels
plt.legend(loc='center right', prop={'size': 15})
plt.title('Employment Rates by Ethnic Groups')
plt.xlabel('Percentages')
plt.ylabel('Boroughs')
plt.rcParams['figure.figsize'] = (30,20)

#Displaying the plot
plt.show()

Above, are the final 3 ethnic groups. For Black/Black British, Barnet is the borough with the highest employment rate with only Bromley trailing behind. Pakistan/Bangladesh has the highest employment rate in Lambeth, but I mentioned earlier, that part of the data is misleading because there was't any record for number of employed  Pakistan/Bangladesh divided by their total population. I selected the second largest percetage of employment rate for the Pakistan/Bangladesh and used it to conclude on where they have their highest employment rate. That borough is, Havering. Other ethnic groups have the highest employment rate in Lambeth with Bexley trailing behind.

**Employment Industries**

My aim is to discover the boroughs that have the highest employment industry across the boroughs in london. What is the industry that has the higest employment and what borough is it located in? Just like I did for the employment rate dataset, I will split the employment industries graphs into two for better analysis.

In [23]:
#Ploting half of the employment industry
plt.barh(employmentindustry.Area, employmentindustry['Agriculture, Forestry & Fishing'], 
         label='Agriculture, Forestry & Fishing')

plt.barh(employmentindustry.Area, employmentindustry['Mining, Quarrying & Utilities'], 
         label='Mining, Quarrying & Utilities')

plt.barh(employmentindustry.Area, employmentindustry['Manufacturing'], 
         label='Manufacturing')

plt.barh(employmentindustry.Area, employmentindustry['Construction'], 
         label='Construction')

plt.barh(employmentindustry.Area, employmentindustry['Motor Trades'], 
         label='Motor Trades')

plt.barh(employmentindustry.Area, employmentindustry['Wholesale'], 
         label='Wholesale')

plt.barh(employmentindustry.Area, employmentindustry['Retail'], 
         label='Retail')

plt.barh(employmentindustry.Area, employmentindustry['Accommodation & Food Services'], 
         label='Accommodation & Food Services')

plt.barh(employmentindustry.Area, employmentindustry['Information & Communication'], 
         label='Information & Communication')

plt.barh(employmentindustry.Area, employmentindustry['Professional, Scientific & Technical'], 
         label='Professional, Scientific & Technical')

plt.barh(employmentindustry.Area, employmentindustry['Business Administration & Support Services'], 
         label='Business Administration & Support Services')

plt.barh(employmentindustry.Area, employmentindustry['Public Administration & Defence'], 
         label='Public Administration & Defence')

plt.barh(employmentindustry.Area, employmentindustry['Education'], 
         label='Education')

plt.barh(employmentindustry.Area, employmentindustry['Health'], 
         label='Health')

plt.barh(employmentindustry.Area, employmentindustry['Arts, Entertainment, Recreation & Other Services'], 
         label='Arts, Entertainment, Recreation & Other Services')

#Adding legend and titles
plt.legend(loc='center right', prop={'size': 15})
plt.title('Employment Industries by Boroughs')
plt.xlabel('Total Numbers')
plt.ylabel('Areas')
plt.rcParams['figure.figsize'] = (20,20)

# Display the plot
plt.show()

In this employment industries chart of london boroughs, in the City of London and Westminister the Professional, Scientific & Technical industry according to the bar chart is the highest job sector. That is very interesting, what could be the population by ethinic group or health condition for example in Westminister & City of London. In second place, also in the city of London, Business Administration, is second highest employment industry. Also in Westminister, the third highest employment industry is Mining, Quaring & Utilities. There is a trend here, the City of London & Westminister have a high rate of a variety of different employment industries, therefore this indicates that there would be more job oppotunities in those boroughs. 
Compared to my findings above, Black/Black British have a high employment rate in Barnet, and looking at this employment industry chart, its safe to assume that Black/Black British are employed in the Health sector. In addition, the goegraphical graphs I ploted above on london census, you discover that there is a high female population in Barnet.

**Correlation Analysis**

I will perform correlation analysis to quantify and amount of linear correspondance between two variables. The two variables are going to be population by bad health and population by christian religion. Those two variables were chosen because when I graphed them geographically earlier on in the notebook, they had a wide spread across all the boroughs in london. I will first perform a Pearson's correlation, then Spearman, plot a scatter graph to find any differences or relationships, I will also use the function corr() to produce a data frame of all the correlation coefficients for each pair of numeric variables. The corr() function is basically a matrix with variables as columns and then it also produces rows that will be very useful to plot a heatmap in the Seaborne library. I will use the Seaborn heatmap hue as a visual indication of positive or negative correlation.

In [24]:
#Selecting the two variables
val1 = ldncensusdf['Population by religion: Christian']
val2 = ldncensusdf['Population by health condition: Bad health']

**Pearson's correlation**

In [25]:
corrPearson, pValPearson = stats.pearsonr(val1,val2)
print ("Pearson's correlation is", corrPearson, "with a p-value of",pValPearson)

**Spearman's correlation**

In [26]:
corrSpearman, pValSpearman = stats.spearmanr(val1,val2)
print ("Spearman's correlation is", corrSpearman, "with a p-value of",pValSpearman)

In [27]:
plt.suptitle('Christians vs Bad health')
plt.xlabel('Christians')
plt.ylabel('Bad Health')
plt.scatter(val1,val2 , c = "#8b0000", s = 30, alpha = 0.5, linewidth='0')

Just like my Pearsons and Spearmans correlation calculated, in the scatter plot above, there doesn't seem to be any linear relationship at all because since my calculated correlation values are between 0 & 0.3, it indicates a weak positive(negative) linear relationship. Despite that, if you keep having a look at the scatter plot over and over again it may look like a regression line could be drawn across.

In [28]:
corr = ldncensusdf.corr()
#print(corr.head())

plt.figure(figsize = (30,30))
import seaborn as sns
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

In [29]:
#Correlation with sole focus on population with bad health
corrbadhealth=corr[['Population by health condition: Bad health']]

#Sorting the amount of correlation
corrbadhealth=corrbadhealth.sort_values(by ='Population by health condition: Bad health',ascending=False)

plt.figure(figsize = (10,20))
import seaborn as sns
ax = sns.heatmap(
    corrbadhealth, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

The correlation above shows the variables that are highly correlated with Bad health. It is very helpful because I can chose those variables to build a model that will be a very good predictor. Now, the next step is to look for variables that have the strongest correlations with Bad health. I will first restructure the correlation matrix in a way that for each row, I will have variables and their correlation listed and then I will sort all the correlations in the right order.

In [30]:
#Make a copy
corrcopy=corr.copy()

#Creating a new column for the variable name & index
corrcopy.index.name = 'var1'
corrcopy.reset_index(inplace=True)

#Preserving the column
corrpair = corrcopy.melt(id_vars = ['var1'])

#Replacing the some of the column names
corrpair = corrpair.rename(columns = {'variable': 'var2','value': 'corr'})

#Removing duplicate rows where var1 and var2 are the same
corrpair = corrpair.drop(corrpair[corrpair['var1']==corrpair['var2']].index)

The next step is to view the distribution of the residuals, locate the top ten most positive & negative correlations and then finally find the variables that correlate with the population of Bath health variable.

In [31]:
#Ploting the histogram
plt.hist(corrpair['corr'],20)
plt.title("Correlation distributions bounded by combinational numerical variables")

#Sorting the correlation rows
corrpair = corrpair.sort_values(by ='corr',ascending=False)

print()
print("Top ten most positively correlated pairs of variables")
print(corrpair.head(10))

print()
print("Top ten most negatively correlated pairs of variables (reverse order)")
print(corrpair.tail(10))

#Adding extra column with absolute correlation
corrpair['abscorr']=abs(corrpair['corr'])
#Sorting in ascending order
corrpair=corrpair.sort_values(by ='abscorr',ascending=False)

print()
print("Top ten most correlated pairs of variables with 'Population by health condition: Bad health'")
print(corrpair[corrpair['var1']=='Population by health condition: Bad health'].head(10))

**Regression analysis**

In order to model the relationship between the population spread of bad health in london and the christian population, I would need to build a couple of regression models. My dependent variable is going to be "Population by health condition: Bad health" and independent variable is going to be "Population by religion: Christian". There are other independent variable I could use like "Population by health condition: Very bad health" for example which is has the best correlation but if you look at the dataset, the figures recorded in both columns are almost the same. Also it wouldn't be wise to find a relationship between Bad health and Very bad health because they're variables that are very similar in description and numbers which I previously just mentioned. 

In [32]:
#Building linear regression model
#There are 4 values that linear regression function returns that describes the model
slope, intercept, rvalue, pvalue, stderr = stats.linregress(val1, val2)

print ("Slope: ", slope)
print ("Intercept: ", intercept)
print ("p_value: ", pvalue)
print ("std_err: ", stderr)

In [33]:
#Calculating regression line
#Calculating 2 points from the parameters
x1=val1.min() #min value of the independent variable
x2=val1.max() #max value of the independent variable
y1=x1*slope + intercept #calculate the dependent variable value from x1
y2=x2*slope + intercept #calculate the dependent variable value from x2

#Ploting scatterplot
plt.suptitle('Christians vs. Bad Health')
plt.xlabel('Population by religion: Christian')
plt.ylabel('Population by health condition: Bad health')
plt.scatter(val1, val2, c = "#D06B36", s = 30, alpha = 0.6, linewidth='0')

plt.plot([x1,x2],[y1,y2])

Looking at the scatterplot, it is obvious that the scatterplot does not seem to fit well due to the fact that there isn't a relationship. I will try fitting a polynomial curve(second order curve) using a numpy function called np.polyval().

In [34]:
#Fiting second-order curve
pCoeff = np.polyfit(val1, val2, 2)
#Sorting the curve to make it look like a curve is supposed to be
xs = np.sort(val1)
#Calculating the value for each use of the polynomial
ys = np.polyval(pCoeff, xs)

#Ploting data values
plt.scatter(val1,val2 , c = "#D06B36", s = 30, alpha = 0.6, linewidth='0')
#Ploting linear "curve" so that it looks dashed
plt.plot([x1,x2],[y1,y2], "k--")
#Ploting second order polynomal curve
plt.plot(xs, ys, linewidth = 3, fillstyle="none")

**Multiple Regression**

In order to carry out a multiple regression I will use the statsmodels library. Following that, I will also calculate, manually, some of the measures. I cannot use my highest correlated variable from above, first of all the recorded data is almost the same which is why it has a 98% correlation with Bad Health. Because my top three highest correlated variable are all within the same topic, health, I will use 'Population by sex: Male' for the multiple regression calculation.

In [35]:
#Using the dependent variable 'Population by health condition: Bad health' here is  is still
dependent = val2;
#The two independent variables(regressors) 
independents = ldncensusdf[["Population by religion: Christian", "Population by sex: Male"]]
#Adding the intercepts
indeptswithconst = sm.add_constant(independents)

# we use the OLS function from statsmodels
model = sm.OLS(dependent, indeptswithconst)
results = model.fit()

print (results.summary())

The model has a 0.438 R-Squared value indicating that the model was not a good fit at all. The R-Squared value is important because it is a measure of how good the regression line approximates the data points. Also, because the P-Value is higher than the coefficient, it signifies a non exiestent relationship. Finally population has a bigger significance on the model because like we calculated earlier, it has a high correlation with Bad health.

**Calculating Residuals**

In [36]:
#Calculating dependent variable values from parameters directly
modelledDependentOLSDirect  = ldncensusdf["Population by religion: Christian"]*results.params["Population by religion: Christian"] + ldncensusdf["Population by sex: Male"]*results.params["Population by sex: Male"] + results.params['const']

#Calculating dependent variable values from the predict method
modelledDependentOLSPredict  = results.predict()

#Calculate the residuals 3 ways
residualsDirect=dependent-modelledDependentOLSDirect
residualsPredict=dependent-modelledDependentOLSDirect
residualsFromLib=results.resid


#both residual distributions are the same (because both methods of
#calculating the dependent variable values are the same)
f, ax = plt.subplots(1, 3)
ax[0].set_title("Residuals (direct)")
ax[0].hist(residualsDirect,20)
ax[1].set_title("Residuals (predict())")
ax[1].hist(residualsPredict,20)
ax[2].set_title("Residuals (from residuals)")
ax[2].hist(residualsFromLib,20)

print(results.params)

Since the residuals look fairly big, I think its safe to say that the model is averagely good.

In [37]:
#Checking standard deviation for more clarity
print(dependent.std())
plt.boxplot(dependent)

With a standard deviation of 500, it is fair to say that the residuals indicated a more than average amount of the variation in the data. From the geographical maps I plotted earlier, you could see how widely spread both variables that I used are.

In [38]:
print("R-squared:",results.rsquared)
print("MSE model:",results.mse_model)
print("MSE residuals:",results.mse_resid)
print("MSE total:",results.mse_total)

In conclusion a R squared score of 0.438, doesn't entirely mean the model is bad because you can gather information from the predictors. I will conclude and finalize by agreeing that my R square score is a fairly good or you could say, reasonable model. Thank you and I hope you find my analysis helpful!